In [12]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# API 키 정보 로드
load_dotenv()

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangGraph-Memory


In [13]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()

In [14]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_ollama import ChatOllama
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]

########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
tool = TavilySearch(max_results=3)
tools = [tool]
# LLM 초기화
llm = ChatOllama(model="llama3.1:8b")

########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm.invoke(state["messages"])]}

# 상태 그래프 생성
graph_builder = StateGraph(State)
# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)
# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[tool])
# 도구 노드 추가
graph_builder.add_node("tools", tool_node)
# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")
# START > chatbot
graph_builder.add_edge(START, "chatbot")
# chatbot > END
graph_builder.add_edge("chatbot", END)

In [15]:
graph = graph_builder.compile(checkpointer=memory)

In [16]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

In [17]:
# 첫 질문
question = (
    "내 이름은 `PETER` 입니다. 한국에 살고 있는 개발자에요. 만나서 반가워요."
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

안녕하세요! 반갑습니다. PETER님, 한국에서 개발을 하시는 분인가 보네요. 관심사가 뭔지 궁금합니다.


In [18]:
# 이어지는 질문
question = "내 이름이 뭐라고 했지?"

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

네!你的 이름은 `PETER` 였습니다. 한국에 사는 개발자 인 것 같았어요!


In [19]:
from langchain_core.runnables import RunnableConfig

question = "내 이름이 뭐라고 했지?"

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "2"},  # 스레드 ID 설정
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        value["messages"][-1].pretty_print()

================================== Ai Message ==================================

우리가 상호 작용을 시작하기 전에, 내 이름은 'Llama'로 줄여서 말할 수도 있는 'Large Language Model Meta AI'입니다.


### Snapshot

In [26]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)
# 그래프 상태 스냅샷 생성
snapshot = graph.get_state(config)
snapshot.values

{'messages': [HumanMessage(content='내 이름은 `PETER` 입니다. 한국에 살고 있는 개발자에요. 만나서 반가워요.', additional_kwargs={}, response_metadata={}, id='1e782b4b-f2a1-4eb8-b735-860752530729'),
  AIMessage(content='안녕하세요! 반갑습니다. PETER님, 한국에서 개발을 하시는 분인가 보네요. 관심사가 뭔지 궁금합니다.', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-01-24T04:21:22.561834Z', 'done': True, 'done_reason': 'stop', 'total_duration': 19555113958, 'load_duration': 13936329375, 'prompt_eval_count': 36, 'prompt_eval_duration': 2993000000, 'eval_count': 31, 'eval_duration': 2623000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-04226ec7-c95a-4372-b105-b88d7d8a4b96-0', usage_metadata={'input_tokens': 36, 'output_tokens': 31, 'total_tokens': 67}),
  HumanMessage(content='내 이름이 뭐라고 했지?', additional_kwargs={}, response_metadata={}, id='930b6d71-c9bb-470b-bfbb-0ac84df356f1'),
  AIMessage(content='네!你的 이름은 `PETER` 였습니다. 한국에 사는 개발자 인 것 같았어요!', additional_kwargs={},

In [21]:
snapshot.config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1efda0b1-cc4f-6020-8004-6008a076c003'}}

In [22]:
snapshot.next

()

In [24]:
from langchain_teddynote.messages import display_message_tree

# 메타데이터(tree 형태로 출력)
display_message_tree(snapshot.metadata)


    source: "loop"
    writes:
        chatbot:
            messages:
                index [0]
                    content: "네!你的 이름은 `PETER` 였습니다. 한국에 사는 개발자 인 것 같았어요!"
                    additional_kwargs: {}
                    response_metadata:
                        model: "llama3.1:8b"
                        created_at: "2025-01-24T04:24:33.384395Z"
                        done: True
                        done_reason: "stop"
                        total_duration: 6790776416
                        load_duration: 928119541
                        prompt_eval_count: 84
                        prompt_eval_duration: 3305000000
                        eval_count: 26
                        eval_duration: 2189000000
                        message: {"role": "assistant", "content": "", "images": None, "tool_calls": None}
                    type: "ai"
                    name: None
                    id: "run-8b2211ee-117b-403e-ac42-07de15196a4e-0"
                    example: 